In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!nohup ollama serve > output.log 2>&1 &

In [3]:
!ollama pull gemma3:latest

In [4]:
!ollama list

NAME             ID              SIZE      MODIFIED       
gemma3:latest    a2af6cc3eb7f    3.3 GB    40 minutes ago    


In [10]:
import requests

def run_ollama(prompt, model):
  response = requests.post("http://localhost:11434/api/generate", json={
      "model": model,
      "prompt": prompt,
      "stream": False
  })
  return response.json()["response"]

In [11]:
prompt = "Translate this sentence to French: 'I am learning about Large Language Models.'"

print(run_ollama(prompt, 'gemma3:latest'))


Here are a few options for translating "I am learning about Large Language Models" into French, with slight nuances:

**1. (Most common and straightforward):**

*   **J'apprends sur les grands modèles de langage.** 

    *   `J'apprends` - I am learning
    *   `sur` - about/on
    *   `les grands modèles de langage` - Large Language Models


**2. (Slightly more formal):**

*   **Je suis en train d'apprendre sur les grands modèles de langage.**

    *   `Je suis en train de` - I am in the process of/I am currently
    *   The rest is the same as above.  This emphasizes the ongoing nature of the learning.

**3. (Focusing on the study):**

*   **Je suis en train d'étudier les grands modèles de langage.**

    *   `étudier` - to study
    *   Again, `en train de` emphasizes the ongoing nature.

**Recommendation:**

For most situations, the first option, **"J'apprends sur les grands modèles de langage,"** is perfectly suitable and the most natural-sounding. 

Would you like me to provide a

# Few shot prompting

In [12]:
few_shot_prompt = """
Translate the following English sentences to French.

English: Hello, how are you?
French: Bonjour, comment ça va?

English: I love machine learning.
French: J'adore l'apprentissage automatique.

English: I am learning about Large Language Models.
French:
"""

print(run_ollama(few_shot_prompt, 'gemma3:latest'))

French: J'apprends sur les grands modèles de langage. 

(Or, a slightly more natural phrasing would be: J'étudie les grands modèles de langage.)



# Chain of Thought Prompting

In [13]:
cot_prompt = """
Question: If I have 3 red balls and 2 blue balls, and I take away 1 red ball, how many red balls do I have left?

Let's think step-by-step.
"""

print(run_ollama(cot_prompt, 'gemma3:latest'))

1. **Start:** You begin with 3 red balls.
2. **Take away:** You remove 1 red ball.
3. **Calculate:** 3 - 1 = 2

**Answer:** You have 2 red balls left.



In [17]:
!pip install ollama

### AI Assistant through Pydantic

In [20]:
from pydantic import BaseModel, Field
from typing import List
import ollama

class FAQEntry(BaseModel):
  question: str = Field(..., description="The FAQ Question")
  answer: str = Field(..., description="The FAQ Answer")

class FAQData(BaseModel):
  faqs: List[FAQEntry]

  def format_for_prompt(self) -> str:
    return "\n".join([f"Q: {faq.question}\nA: {faq.answer}" for faq in self.faqs])

#Sample FAQ Data
faq_data = FAQData(
    faqs = [
        FAQEntry(question="What is an AI Agent?", answer="An AI agent is a system that can autonomously perform tasks based on inputs and learned by knowledge"),
        FAQEntry(question="How can AI agents improve efficiency?", answer="AI agents can automate repetitive tasks, streamline decision-making, and enhance customer support with accurate responses."),
        FAQEntry(question="What industries benefit most from AI agents?", answer="Industries like healthcare, e-commerce, finance, and customer support heavily benefit from AI agents."),
        FAQEntry(question="Are AI agents customizable?", answer="Yes, AI agents can be trained with specific data and prompts to suit various use cases.")
    ]
)

#Chatbot Function
def chatbot():
  print("Welcome to the AI FAQ Chatbot (gemma3 via Ollama + Pydantic)! Ask about agents and their benefits. Type 'exit' to quit.")
  faq_context = faq_data.format_for_prompt()
  while True:
    user_input = input("You: ").strip()
    if user_input.lower() == 'exit':
      print("Goodbye!")
      break

    #Build a prompt
    prompt = f"""You are an AI Chatbot. Answer shortly and precisely to the point. Answer this question based on the provided faq data:
    {faq_context}
    Question: {user_input}
    Answer: """
    response = ollama.chat(model="gemma3:latest", messages=[{'role': 'user', 'content': prompt}])
    print("Bot: ",response["message"]["content"])

# Call the chatbot function
display(chatbot())

Welcome to the AI FAQ Chatbot (gemma3 via Ollama + Pydantic)! Ask about agents and their benefits. Type 'exit' to quit.
You: hi
Bot:  Hello.
You: How are you?
Bot:  I am a system that can autonomously perform tasks based on inputs and learned by knowledge.
You: give 3 steps to learn system design precisely
Bot:  1. Understand core concepts.
2. Practice designing systems.
3. Study existing designs.
You: exit
Goodbye!


None